In [34]:
#Importing the required functions
import pandas as pd
import numpy as np
import regex as re
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc
from sklearn.svm import SVC
import warnings
warnings.filterwarnings("ignore")
import random
import mlflow
import logging
from urllib.parse import urlparse
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


**Reading in the train, test and validation data**

In [19]:
path_train = './data/train.csv'
path_val = './data/validation.csv'
path_test = './data/test.csv'
path_mod_df = './data/mod_df.csv'

train_df = pd.read_csv(path_train)
val_df = pd.read_csv(path_val)
test_df = pd.read_csv(path_test)
raw_data = pd.read_csv(path_mod_df)

In [20]:
train_df['y_train'] = train_df['y_train'].map({'ham': 1, 'spam': 0})
val_df['y_val'] = val_df['y_val'].map({'ham': 1, 'spam': 0})
test_df['y_test'] = test_df['y_test'].map({'ham': 1, 'spam': 0})

**Creating the bag of words transformer**

In [22]:
vectorizer = CountVectorizer()
vectorizer.fit(train_df.X_train)

bow_transformer = vectorizer.vocabulary_
print(len(bow_transformer))

7340


**Converting all the data to be used into bag of words form**

In [23]:
X_train = vectorizer.transform(train_df.X_train)
X_val = vectorizer.transform(val_df.X_val)
X_test = vectorizer.transform(test_df.X_test)

print(X_train.shape, X_val.shape, X_test.shape)

(4514, 7340) (502, 7340) (558, 7340)


**Creating the tf-idf transformer**

In [24]:
tfidf_transformer = TfidfTransformer().fit(X_train)

**Converting the data into tf-idf form**

In [25]:
tfidf_X_train = tfidf_transformer.transform(X_train)
tfidf_X_val = tfidf_transformer.transform(X_val)
tfidf_X_test = tfidf_transformer.transform(X_test)

print(tfidf_X_train.shape, tfidf_X_val.shape, tfidf_X_test.shape)

(4514, 7340) (502, 7340) (558, 7340)


**Multinomial Naive Bayes Model based on tf-idf tokenizer**

**Building a function to create the spam detection model and compute the evaluation metrics for the predicted values based on the test set**

In [26]:
def MNB_model(alp):
    spam_detection_model = MultinomialNB(alpha = alp).fit(tfidf_X_train, train_df.y_train)
    test_predictions = spam_detection_model.predict(tfidf_X_test)

    acc_sc = accuracy_score(test_df.y_test, test_predictions)
    
    #Computing Precision and Recall
    precision, recall, thresholds = precision_recall_curve(test_df.y_test, test_predictions)
    # Use AUC function to calculate the area under the curve of precision recall curve
    aupcr = auc(recall, precision)
    
    conf_m = confusion_matrix(test_df.y_test, test_predictions)

    #print("The AUPCR score is:",aupcr)
    return [spam_detection_model, acc_sc, aupcr, conf_m]

**Logistic Regression Model based on tf-idf tokenizer**

**Creating the spam detection model and computing the evaluation metrics for the predicted values based on the test set**

In [27]:
def Log_model(C):
    spam_detection_model_2 = LogisticRegression(C = i)
    spam_detection_model_2.fit(tfidf_X_train, train_df.y_train)
    test_predictions = spam_detection_model_2.predict(tfidf_X_test)
    acc_sc = accuracy_score(test_df.y_test, test_predictions)
    
    #Computing Precision and Recall
    precision, recall, _ = precision_recall_curve(test_df.y_test, test_predictions)
    # Use AUC function to calculate the area under the curve of precision recall curve
    aupcr = auc(recall, precision)
    #print("The AUPCR score is:",aupcr)

    conf_m = confusion_matrix(test_df.y_test, test_predictions)

    #print("The AUPCR score is:",aupcr)
    return [spam_detection_model_2, acc_sc, aupcr, conf_m]

**Support Vector Classifier Model**

**Creating the spam detection model and computing the evaluation metrics for the predicted values based on the test set**

In [43]:
def SVC_Model(C):    
    spam_detection_model_3 = SVC(C = i)
    spam_detection_model_3.fit(tfidf_X_train, train_df.y_train)
    test_predictions = spam_detection_model_3.predict(tfidf_X_test)
    acc_sc = accuracy_score(test_df.y_test, test_predictions)
    
    #Computing Precision and Recall
    precision, recall, _ = precision_recall_curve(test_df.y_test, test_predictions)
    # Use AUC function to calculate the area under the curve of precision recall curve
    aupcr = auc(recall, precision)
    #print("The AUPCR score is:",aupcr)

    conf_m = confusion_matrix(test_df.y_test, test_predictions)

    #print("The AUPCR score is:",aupcr)
    return [spam_detection_model_3, acc_sc, aupcr, conf_m]

In [29]:
from  mlflow.tracking import MlflowClient
client = MlflowClient()
#log into MLflow

#Set storage directory
mlflow.set_tracking_uri('\mlruns')

#set experiment
mlflow.set_experiment('SMS Spam Classification Model Evaluation')

<Experiment: artifact_location='\\mlruns/894247250466392331', creation_time=1677526330970, experiment_id='894247250466392331', last_update_time=1677526330970, lifecycle_stage='active', name='SMS Spam Classification Model Evaluation', tags={}>

In [46]:
mlflow.set_tracking_uri('\mlruns')
mlflow.sklearn.autolog()

md_name = "Multinomial Naive Bayes" 

for i in np.arange(0.05, 2.25, 0.25):
    with mlflow.start_run(run_name = f"Model : {md_name}, Alpha : {i}"):
        
        #Log parameters
        mlflow.log_param("Alpha",i)

        #Running the model
        model_metrics = MNB_model(i)

        #Logging metrics
        mlflow.log_metric("Accuracy", model_metrics[1])
        mlflow.log_metric("AUPCR", model_metrics[2])

        #Logging JSON of confusion matrix
        mlflow.log_dict(np.array(model_metrics[3]).tolist(), "confusion_matrix.json")
        mlflow.sklearn.log_model(model_metrics[0], "model")

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        mlflow.sklearn.log_model(
            sk_model = model_metrics[0],
            artifact_path="sklearn-model",
            registered_model_name="Multinomial Naive Bayes"
        )
        if tracking_url_type_store != "file":
            mlflow.sklearn.log_model(model_metrics[0], "model", registered_model_name="Multinomial Naive Bayes")
        else:
          mlflow.sklearn.log_model(model_metrics[0], "model")

        


2023/02/28 02:23:16 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
Registered model 'Multinomial Naive Bayes' already exists. Creating a new version of this model...
2023/02/28 02:23:40 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Multinomial Naive Bayes, version 19
Created version '19' of model 'Multinomial Naive Bayes'.
Registered model 'Multinomial Naive Bayes' already exists. Creating a new version of this model...
2023/02/28 02:23:47 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Multinomial Naive Bayes, version 20
Created version '20' of model 'Multinomial Naive Bayes'.
Registered model 'Multinomial Naive Bayes' already exist

KeyboardInterrupt: 

In [36]:
mlflow.set_tracking_uri('\mlruns')
mlflow.sklearn.autolog()

md_name = "Logistic Regression"

for i in [0.1, 0.5, 1, 10, 20, 50, 100]:
    with mlflow.start_run(run_name = f"Model : {md_name}, C : {i}"):
        
        #Log parameters
        mlflow.log_param("C",i)

        #Running the model
        model_metrics = Log_model(i)

        #Logging metrics
        mlflow.log_metric("Accuracy", model_metrics[1])
        mlflow.log_metric("AUPCR", model_metrics[2])

        #Logging JSON of confusion matrix
        mlflow.log_dict(np.array(model_metrics[3]).tolist(), "confusion_matrix.json")
        mlflow.sklearn.log_model(model_metrics[0], "model")

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        mlflow.sklearn.log_model(
            sk_model = model_metrics[0],
            artifact_path="sklearn-model",
            registered_model_name="Logistic Regression"
        )
        if tracking_url_type_store != "file":
            mlflow.sklearn.log_model(model_metrics[0], "model", registered_model_name = "Logistic Regression")
        else:
          mlflow.sklearn.log_model(model_metrics[0], "model")

        


2023/02/28 01:46:23 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
Successfully registered model 'Logistic Regression'.
2023/02/28 01:46:46 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Logistic Regression, version 1
Created version '1' of model 'Logistic Regression'.
Registered model 'Logistic Regression' already exists. Creating a new version of this model...
2023/02/28 01:46:54 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Logistic Regression, version 2
Created version '2' of model 'Logistic Regression'.
Registered model 'Logistic Regression' already exists. Creating a new version of this model...
2023/02/28 01:47:20 INFO mlflow

In [44]:
mlflow.sklearn.autolog()

md_name = "Support Vector Classifier Model"

for i in [0.1, 0.5, 1, 10, 20, 50, 100]:
    with mlflow.start_run(run_name = f"Model : {md_name}, C : {i}"):
        
        #Log parameters
        mlflow.log_param("C",i)

        #Running the model
        model_metrics = SVC_Model(i)

        #Logging metrics
        mlflow.log_metric("Accuracy", model_metrics[1])
        mlflow.log_metric("AUPCR", model_metrics[2])

        #Logging JSON of confusion matrix
        mlflow.log_dict(np.array(model_metrics[3]).tolist(), "confusion_matrix.json")
        mlflow.sklearn.log_model(model_metrics[0], "model")

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        mlflow.sklearn.log_model(
            sk_model = model_metrics[0],
            artifact_path="sklearn-model",
            registered_model_name = "Support Vector Classifier Model"
        )
        if tracking_url_type_store != "file":
            mlflow.sklearn.log_model(model_metrics[0], "model", registered_model_name = "Support Vector Classifier Model")
        else:
          mlflow.sklearn.log_model(model_metrics[0], "model")

        


2023/02/28 02:09:44 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
Successfully registered model 'Support Vector Classifier Model'.
2023/02/28 02:10:16 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Support Vector Classifier Model, version 1
Created version '1' of model 'Support Vector Classifier Model'.
Registered model 'Support Vector Classifier Model' already exists. Creating a new version of this model...
2023/02/28 02:10:23 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Support Vector Classifier Model, version 2
Created version '2' of model 'Support Vector Classifier Model'.
Registered model 'Support Vector Classifier Model' alr

In [95]:
mlflow.set_experiment('SMS Spam Classification Model Evaluation')
#Running the models and logging the runs with MLFlow
md_name = "Multinomial Naive Bayes" 
for i in np.arange(0.05, 2.25, 0.25):
    with mlflow.start_run() as run: #inside brackets run_name='test'
        #Log parameters
        mlflow.log_param("Model",md_name)
        mlflow.log_param("Alpha",i)
        #Running the model
        model_metrics = MNB_model(i)
        #Logging metrics
        mlflow.log_metric("Accuracy", model_metrics[0])
        mlflow.log_metric("AUPCR", model_metrics[1])

md_name = "Logistic Regression"
for i in [0.1, 0.5, 1, 10, 20, 50, 100]:
    with mlflow.start_run() as run: 
        #Log parameters
        mlflow.log_param("Model",md_name)
        mlflow.log_param("C",i)
        #Running the model
        model_metrics = Log_model(i)
        #Logging metrics
        mlflow.log_metric("Accuracy", model_metrics[0])
        mlflow.log_metric("AUPCR", model_metrics[1])


md_name = "Support Vector Classifier"
for i in [0.1, 0.5, 1, 10, 20, 50, 100]:
    with mlflow.start_run() as run: 
        #Log parameters
        mlflow.log_param("Model",md_name)
        mlflow.log_param("Alpha",i)
        #Running the model
        model_metrics = MNB_model(i)
        #Logging metrics
        mlflow.log_metric("Accuracy", model_metrics[0])
        mlflow.log_metric("AUPCR", model_metrics[1])
        
